In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

The SSP encoding method (a.k.a. fractional power encoding, fractional binding, etc.) can be thought of as a single hidden layer of neurons whose activation functions are the `sin` function.  Here's how we can build that.

We start with the normal encoding method we have been using:

In [2]:
N = 2
D = 32

import sspspace
encoder = sspspace.RandomSSPSpace(ssp_dim=D, domain_dim=2)

x = np.array([3,1.2])
x_ssp = encoder.encode(x)
print(x_ssp)

[[ 0.1043889  -0.08546952  0.15821391  0.19910652  0.21500024 -0.2994783
  -0.08785525  0.25473042  0.08738421  0.18838902 -0.13542383 -0.1047516
  -0.05760779 -0.01684594  0.15328399 -0.08690657  0.2282585   0.20040973
   0.00719742 -0.2614392   0.04092408 -0.43821752  0.03885155 -0.03288524
   0.03185555 -0.03305588  0.08908084  0.00407347  0.27975942  0.22905925
  -0.15331175  0.28328137]]


Now let's do that manually, rather than using the `encode` method, just to make sure we know what's going on.  

The formula is $\phi(x) = \mathcal{F}^{-1}\{e^{i\theta x}\}$ so let's go ahead and implement that, remembering that $e^{i\theta} = cos(\theta)+i sin(\theta)$.

In [3]:
phase = encoder.phase_matrix @ x
x_ssp2 = np.fft.ifft(np.cos(phase)+1.0j*np.sin(phase)).real
print(x_ssp2)
assert np.allclose(x_ssp, x_ssp2)

[ 0.1043889  -0.08546952  0.15821391  0.19910652  0.21500024 -0.2994783
 -0.08785525  0.25473042  0.08738421  0.18838902 -0.13542383 -0.1047516
 -0.05760779 -0.01684594  0.15328399 -0.08690657  0.2282585   0.20040973
  0.00719742 -0.2614392   0.04092408 -0.43821752  0.03885155 -0.03288524
  0.03185555 -0.03305588  0.08908084  0.00407347  0.27975942  0.22905925
 -0.15331175  0.28328137]


The Fourier transform and its inverse are linear operations, so we can just generate the inverse Discrete Fourier Transform matrix and use that instead.

In [4]:
phase = encoder.phase_matrix @ x
idft = np.fft.ifft(np.eye(D))
x_ssp3 = (idft @ (np.cos(phase)+1.0j*np.sin(phase))).real
print(x_ssp3)
assert np.allclose(x_ssp, x_ssp3)

[ 0.1043889  -0.08546952  0.15821391  0.19910652  0.21500024 -0.2994783
 -0.08785525  0.25473042  0.08738421  0.18838902 -0.13542383 -0.1047516
 -0.05760779 -0.01684594  0.15328399 -0.08690657  0.2282585   0.20040973
  0.00719742 -0.2614392   0.04092408 -0.43821752  0.03885155 -0.03288524
  0.03185555 -0.03305588  0.08908084  0.00407347  0.27975942  0.22905925
 -0.15331175  0.28328137]


The inverse DFT matrix contains complex values, but we know the output is completely real.  So let's rewrite this to get rid of the use of complex numbers.

In [5]:
phase = encoder.phase_matrix @ x
cos = np.cos(phase)
sin = np.sin(phase)
x_ssp4 = cos@idft.real - sin@idft.imag
print(x_ssp4)
assert np.allclose(x_ssp, x_ssp4)

[ 0.1043889  -0.08546952  0.15821391  0.19910652  0.21500024 -0.2994783
 -0.08785525  0.25473042  0.08738421  0.18838902 -0.13542383 -0.1047516
 -0.05760779 -0.01684594  0.15328399 -0.08690657  0.2282585   0.20040973
  0.00719742 -0.2614392   0.04092408 -0.43821752  0.03885155 -0.03288524
  0.03185555 -0.03305588  0.08908084  0.00407347  0.27975942  0.22905925
 -0.15331175  0.28328137]


Because of the fact that the output is supposed to be real, there are symmetries in the phase matrix.  Because of this, we can actually just use the first half of the weight matrix.  This cuts the number of sines and cosines we have to compute in half.

In [6]:
phase = encoder.phase_matrix[:D//2] @ x
cos = np.cos(phase)
sin = np.sin(phase)

assert D % 2 == 0   # we're assuming D is even in order to simplify the math here

w = idft[D//2:].real.copy()
w[1:] = w[::-1][:-1]
w_cos = idft[:D//2].real + w

w = idft[D//2:].imag.copy()
w[1:] = -w[::-1][:-1]
w_sin = idft[:D//2].imag + w

x_ssp5 = cos @ w_cos - sin @ w_sin
print(x_ssp5)
assert np.allclose(x_ssp, x_ssp5)



[ 0.1043889  -0.08546952  0.15821391  0.19910652  0.21500024 -0.2994783
 -0.08785525  0.25473042  0.08738421  0.18838902 -0.13542383 -0.1047516
 -0.05760779 -0.01684594  0.15328399 -0.08690657  0.2282585   0.20040973
  0.00719742 -0.2614392   0.04092408 -0.43821752  0.03885155 -0.03288524
  0.03185555 -0.03305588  0.08908084  0.00407347  0.27975942  0.22905925
 -0.15331175  0.28328137]


Now let's try implementing this network in Nengo, just to force it to be a completely normal neural network.

In [7]:
import nengo

# define a sine-wave neuron model
class SineNeuron(nengo.RectifiedLinear):
    def step(self, dt, J, output):
        output[...] = np.sin(J)

model = nengo.Network()
with model:
    # an input with no non-linearity
    stim = nengo.Node(x)
    
    # create a hidden layer with a sin() function for activation.  For the first D/2 of them,
    #  we add a bias of pi/2 so that those ones are computing cos() instead of sin()
    ssp_f = nengo.Ensemble(n_neurons=D, dimensions=1, 
                           neuron_type=SineNeuron(),
                           gain=nengo.dists.Choice([1]), 
                           bias=np.concatenate([np.ones(D//2)*np.pi/2,np.zeros(D//2)]))
    
    # connect the inputs to the hidden layer, setting the connection weights to compute the phase matrix
    nengo.Connection(stim, ssp_f.neurons, transform=np.concatenate([encoder.phase_matrix[:D//2], 
                                                                    encoder.phase_matrix[:D//2]]), synapse=None)
    
    # define the output layer (also without a nonlinearity)
    ssp = nengo.Node(None, size_in=D)
    # connect the output weights to compute the iDFT
    nengo.Connection(ssp_f.neurons, ssp, transform=np.concatenate([w_cos, -w_sin]).T, synapse=None)
    
    p = nengo.Probe(ssp)
    
sim = nengo.Simulator(model, progress_bar=False)
with sim:
    sim.run(0.001)
x_ssp6 = sim.data[p][-1]
print(x_ssp6)
assert np.allclose(x_ssp, x_ssp6)

[ 0.1043889  -0.08546952  0.15821391  0.19910652  0.21500024 -0.2994783
 -0.08785525  0.25473042  0.08738421  0.18838902 -0.13542383 -0.1047516
 -0.05760779 -0.01684594  0.15328399 -0.08690657  0.2282585   0.20040973
  0.00719742 -0.2614392   0.04092408 -0.43821752  0.03885155 -0.03288524
  0.03185555 -0.03305588  0.08908084  0.00407347  0.27975942  0.22905925
 -0.15331175  0.28328137]


And for the sake of completeness, let's also implement this in Nengo OCL.  This requires writing an OpenCL implementation of the sine wave neuron type.  The code for that is just `outR = sin(J);` and the rest of the below code is just the hooks needed to tell `nengo_ocl` about the new neuron type.

In [9]:
from nengo_ocl.utils import as_ascii
from mako.template import Template   
from nengo_ocl.clra_nonlinearities import _plan_template

def plan_sine_neuron(queue, J, outR, **kwargs):
    inputs = dict(J=J)
    outputs = dict(outR=outR)
    parameters = dict()
    textconf = dict(type=J.ctype)

    decs = """
        """
    text = """
        outR = sin(J);
        """
    decs = as_ascii(Template(decs, output_encoding='ascii').render(**textconf))
    text = as_ascii(Template(text, output_encoding='ascii').render(**textconf))
    cl_name = "cl_linear"
    return _plan_template(
        queue, cl_name, text, declares=decs,
        inputs=inputs, outputs=outputs, parameters=parameters, **kwargs) 

import nengo_ocl
class SSPSimulator(nengo_ocl.Simulator):
    def _plan_SineNeuron(self, ops):
        J = self.all_data[[self.sidx[op.J] for op in ops]]
        R = self.all_data[[self.sidx[op.output] for op in ops]]
        return [plan_sine_neuron(self.queue, J, R)]
    
sim = SSPSimulator(model, progress_bar=False)
with sim:
    sim.run(0.001)
    
x_ssp7 = sim.data[p][-1]
print(x_ssp7)
assert np.allclose(x_ssp, x_ssp7, rtol=1e-4)  # nengo_ocl uses float32, not float64, so increase the tolerance

[ 0.10438887 -0.08546952  0.15821394  0.19910657  0.21500024 -0.29947835
 -0.08785525  0.2547304   0.08738425  0.18838897 -0.13542378 -0.10475165
 -0.05760778 -0.01684602  0.153284   -0.08690657  0.22825852  0.20040962
  0.00719745 -0.26143906  0.04092405 -0.43821752  0.03885158 -0.03288527
  0.03185548 -0.0330558   0.08908083  0.00407355  0.2797594   0.22905919
 -0.1533118   0.28328145]
